<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    gbvs Mini-Challenge 1 <br> 
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Si Ben Tran im HS 2023.<br>Bachelor of Science FHNW in Data Science.
</p>

In dieser Mini-Challenge werden LE1 und LE2 von gbsv geprüft. Jede:r Studierende:r hat eine individualisierte Aufgabenstellung. Die Abgabe soll ebenfalls einzigartig sein. Dazu wird in dieser Mini-Challenge ein Steckbrief über ein Land in Form von Experimenten in Bild- und Signalverarbeitung zusammengestellt. Das heisst, du wählst passend zu deinem gewählten Land (siehe Google Docs) und den dir zugeordneten Aufgaben Bilder und Signale aus. Wenn du z.B. die Schweiz hast, könntest du Löcher in Bildern vom Emmentalerkäse finden. Die Programmiersprache und die Code-Dokumentation darf frei gewählt werden. Sofern nicht anders erwähnt, dürfen vorhandene Bibliotheken verwendet werden. Gebe die Quellen deiner Daten und ggf. deines Codes an. 

Teil der Mini-Challenge ist es, die Abgabe von jemand anderem zu bewerten. 

Abgaben (Termin siehe Spaces): Code, Resultate und Report gemäss Vorlage. Fürs Peer-Grading sollen die Abgaben anonymisiert sein. 
Google Docs: Ländereinschreibung


# 1 Grundlagen Bild- und Signalverarbeitung (LE1) 

## 1.1 Bildeigenschaften
Suche verschiedene Bilder unterschiedlicher Szenen passend zu deinem Land oder nehme selbst welche auf. Die Bilder sollen sich eignen, um Anpassungen der Bildeigenschaften {'Wertebereiche von Pixeln, Schärfe, Farbräume (Korrektur oder Transformationen)'} in Experimenten zu demonstrieren. Messe dazu zuerst auf deinen Bildern die dir zugeordneten Bildeigenschaften mittels geeigneten Metriken. Definiere danach ein Ziel, wie die dir zugeordneten Bildeigenschaften verändert werden sollen. Definiere ein paar Experimente, um dieses Ziel zu erreichen, und führe die Experimente mit deinen Bildern und geeigneten Methoden aus. Analysiere die Histogramme der ursprünglichen Bilder und während deinen Experimenten. Führe ggf. eine Histogramm-Entzerrung (Englisch: histogram equalization) durch. Diskutiere deine Daten-, Parameter- und Methodenwahl und die erzielten Ergebnisse. 

In [ ]:
# Data Science standard libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Operating System
import os

# image libraries
from PIL import Image
from skimage import color, exposure
from skimage.metrics import peak_signal_noise_ratio


### 1.1.1 Bilder Laden 

In einem ersten Schritt wurden 20 Bilder aus dem Internet heruntergeladen. Die letzten fünf weitere wurden durch Dall-E generiert.   
Gesamthaft befindet sich in unserem Bilder Pool 25 Dateien, die numeriert sind.   
In einem ersten Schritt visualisieren wir die 25 Bilder und wählen dann für jedes Experiment jeweils zwei passende Bilder aus.  
Gesamthaft werden es sechs Bilder sein für die drei Experimente (Wertebereich von Pixeln, Schärfe, Farbräume (Korrektur oder Transformationen)).

In [ ]:
# List of picture filenames
dateinamen = ["1.jpg", "2.jpg", "3.jpg", "4.jpg", "5.jpg", "6.jpg", "7.jpg", "8.jpg", "9.jpg", "10.jpg", "11.jpg", "12.jpg", "13.jpg", "14.jpg", "15.jpg", "16.jpg", "17.jpg", "18.jpg", "19.jpg", "20.jpg", "21.png", "22.png", "23.png", "24.png", "25.png"]

# Create a list to store the images
images = []

# Loop through the filenames and read and store the images in the list
for dateiname in dateinamen:
    # Read the image using PIL
    image = Image.open(f"data/images/{dateiname}")
    images.append(image)

# Create a 5x5 grid for the subplots
fig, axes = plt.subplots(5, 5, figsize=(15, 12))

# Loop through the images and display them with titles
for i, image in enumerate(images):
    # Get image size in pixels
    width, height = image.size

    # Get file size in kilobytes
    file_size_kb = os.path.getsize(f"data/images/{dateinamen[i]}") / 1024

    # Calculate aspect ratio
    aspect_ratio = width / height

    # Choose the appropriate subplot for the current image
    ax = axes[i // 5, i % 5]

    # Display the image on the subplot
    ax.imshow(image)
    ax.axis('off')  # Turn off axis labels

    # Create a title with file name, resolution, and file size
    title = f"{dateinamen[i]} ({width}x{height}, {file_size_kb:.2f} KB)"
    
    # Add the aspect ratio on a new line below the title
    title += f"\nAspect Ratio: {aspect_ratio:.2f}"
    ax.set_title(title, fontsize=10)

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.3, hspace=0.3)

# Set Seaborn style
sns.set_style("whitegrid")

# Show the plot
plt.show()



### 1.1.2 Bilder selektieren & geeignete Metriken

Wir selektieren für jedes Experiment zwei Bilder und messen die geeigneten Bildmetriken im verlaufe der Experimente. 
Hier schreiben wir die Funktionen die wir dafür benötigen.

Folgende Metriken werden als geeignet erachtet: 
RMSE - Root Mean Square Error - Ein Vergleich der Pixelwerte zwischen zwei Bildern.   
Die Histogramme der Bilder mit dem Parameter log=True, um die Verteilung der Pixelwerte vor und nach der Bearbeitung zu sehen. 

In [ ]:
# Selektieren der Bilder und speicher in einer Liste
image_pixel = ["2.jpg", "21.png"]
image_schaerfe = ["10.jpg", "19.jpg"]
image_farbraum = ["11.jpg", "22.png"]

# visualize images in 3x2 grid 
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
for i, image in enumerate(image_pixel + image_schaerfe + image_farbraum):
    ax = axes[i // 2, i % 2]
    ax.imshow(Image.open(f"data/images/{image}"))
    ax.axis('off')
    ax.set_title(image, fontsize=10)
plt.subplots_adjust(wspace=0, hspace=0.1)
sns.set_style("whitegrid")
plt.suptitle("Bilderauswahl für die Experimente \n Wertebereich von Pixeln, Schärfe, Farbräume", fontsize=20)
plt.show()



In [ ]:
# Metriken für die Bilder definieren 
def calc_rmse_images(image1, image2):
    """
    Calculate the Root Mean Square Error (RMSE) between two images.

    Parameters:
    image1 (numpy array): The first image represented as a numpy array
    image2 (numpy array): The second image represented as a numpy array

    Returns:
    float: The RMSE between the two images
    """
    # Check if both images have the same shape
    if image1.shape != image2.shape:
        raise ValueError("Both images must have the same shape")

    # Calculate the RMSE
    rmse = np.sqrt(np.mean((image1 - image2)**2))
    return rmse

def image_and_histo(image1, image2, title1, title2, log_scale=False):
    """
    Plots two images and their RGB histograms in a 2x2 subplot each.

    Parameters:
    - image1, image2: 3D numpy arrays representing RGB images.
    - title1, title2: Titles for the subplots of image1 and image2 respectively.
    - log_scale: Boolean. If True, sets the y-axis of the histogram to logarithmic scale.
    """
    
    # Helper function to plot RGB histogram for an image
    def plot_rgb_histogram(ax, img, log_scale):
        colors = ('r', 'g', 'b')
        total_pixels = img.shape[0] * img.shape[1]
        for i, color in enumerate(colors):
            histogram, bin_edges = np.histogram(
                img[:, :, i], 
                bins=256, 
                range=(0, 255)
            )
            normalized_histogram = histogram / total_pixels  # Normalize histogram
            ax.plot(bin_edges[0:-1], 
                    normalized_histogram, 
                    color=color)
        if log_scale:
            ax.set_yscale('log')  # Set logarithmic scale on y-axis
        ax.set_xlim([0, 255])

    fig, axs = plt.subplots(2, 2, figsize=(12, 6))
    
    # Display image1
    axs[0, 0].imshow(image1)
    axs[0, 0].set_title(title1)

    # Plot RGB histogram for image1
    plot_rgb_histogram(axs[0, 1], image1, log_scale)
    axs[0, 1].set_title(f"{'Log-Scaled ' if log_scale else ''}RGB Histogram for {title1}")

    # Display image2
    axs[1, 0].imshow(image2)
    axs[1, 0].set_title(title2)

    # Plot RGB histogram for image2
    plot_rgb_histogram(axs[1, 1], image2, log_scale)
    axs[1, 1].set_title(f"{'Log-Scaled ' if log_scale else ''}RGB Histogram for {title2}")

    plt.tight_layout()
    plt.show()

### 1.1.3 Ziel definieren

Nachdem wir uns die Eigenschaften der Bilder angeschaut haben, definieren wir ein Ziel für jedes Experiment.

| Experiment | Ziel | Bilder |
| --- | --- | --- |
| Wertebereich von Pixeln | Auf einzelne Pixel Addition und Subtraktion durchführen können. <br> Wertebereich von Bildern Filtern zu können | 2, 21 |
| Schärfe | Bild schärfer und unscharf machen zu können. | 10, 19 |
| Farbräume (Korrektur oder Transformationen) | Helligkeit eines Bildes ändern können. <br> Kontrast eines Bildes verändern. | 11, 22 | 


### 1.1.4 Experiment Wertebereich von Pixeln

In diesem Experiment versuchen wir den Wertebereich des Bildes zu ändern. Dabei versuchen wir in einem ersten Exerperiment durch Addition und Subtraktion der Bilder 2 und 21 das Bild zu verändern und vergleichen anschliessend beide Resultate miteinandner.  


#### 1.1.4.1 Experiment durch Addition / Subtraktion

Wir schreiben eine Funktion, die es uns erlaubt, eine beliebige Zahl auf die Pixelwerte des Bildes zu addieren.   
Pixelwerte die über 255 werden auf 255 geclippt und Pixelwerte die unter 0 fallen werden auf 0 geclippt.   
Die Funktion sollte uns die Möglichkeit geben, auszuwählen, welchen Farbkanal wir verändern möchten.

In [ ]:
def change_value_range(image: np.ndarray, add_r: int, add_g: int, add_b: int) -> np.ndarray:
    """
    Ändert den Wertebereich eines Bildes, indem es eine bestimmte Zahl zu den Pixelwerten der einzelnen Farbkanäle addiert.
    
    Parameters:
    image (np.ndarray): Das Eingabebild als Numpy-Array mit Form (Höhe, Breite, 3).
    add_r (int): Der zu addierende Wert für den Rotkanal.
    add_g (int): Der zu addierende Wert für den Grünkanal.
    add_b (int): Der zu addierende Wert für den Blaukanal.
    
    Returns:
    np.ndarray: Das bearbeitete Bild.
    """
    # Überprüfen, ob das Bild die korrekte Form hat
    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("Das Eingabebild muss die Form (Höhe, Breite, 3) haben.")
    
    # Erstellen einer Kopie des Bildes, um das Originalbild nicht zu verändern
    new_image = image.copy().astype(np.int16)  # Konvertieren zu int16, um Überlauf zu verhindern
    
    # Addieren der Werte zu den entsprechenden Kanälen
    new_image[:, :, 0] += add_r  # Rotkanal
    new_image[:, :, 1] += add_g  # Grünkanal
    new_image[:, :, 2] += add_b  # Blaukanal
    
    # Clipping der Pixelwerte, um sicherzustellen, dass sie im gültigen Bereich bleiben
    np.clip(new_image, 0, 255, out=new_image)
    
    # Konvertieren zurück zu uint8
    new_image = new_image.astype(np.uint8)

    # Rückgabe des bearbeiteten Bildes
    return Image.fromarray(new_image)

In [ ]:
# Wertebereich der Pixel manipulieren
image_2_1 = change_value_range(image = np.array(Image.open("data/images/2.jpg")), 
                               add_r=100, 
                               add_g=100, 
                               add_b=100)

image_2_2 = change_value_range(image = np.array(Image.open("data/images/2.jpg")),
                                add_r=-100,
                                add_g=-100,
                                add_b=-100)

image_21_1 = change_value_range(image = np.array(Image.open("data/images/21.png")),
                                add_r=150,
                                add_g=150,
                                add_b=150)

image_21_2 = change_value_range(image = np.array(Image.open("data/images/21.png")),
                                add_r=-150,
                                add_g=-150,
                                add_b=-150)

# Calculate MSE for the images 
rmse_2_1 = calc_rmse_images(image1 = np.array(Image.open("data/images/2.jpg")), image2 = np.array(image_2_1))
rmse_2_2 = calc_rmse_images(image1 = np.array(Image.open("data/images/2.jpg")), image2 = np.array(image_2_2))
rmse_21_1 = calc_rmse_images(image1 = np.array(Image.open("data/images/21.png")), image2 = np.array(image_21_1))
rmse_21_2 = calc_rmse_images(image1 = np.array(Image.open("data/images/21.png")), image2 = np.array(image_21_2))
# calculate MSE for orginal image
rmse_2 = calc_rmse_images(image1 = np.array(Image.open("data/images/2.jpg")), image2 = np.array(Image.open("data/images/2.jpg")).copy())
rmse_21 = calc_rmse_images(image1 = np.array(Image.open("data/images/21.png")), image2 = np.array(Image.open("data/images/21.png")).copy())

# Calculate PSNR for the images
psnr_2_1 = peak_signal_noise_ratio(np.array(Image.open("data/images/2.jpg")), np.array(image_2_1))
psnr_2_2 = peak_signal_noise_ratio(np.array(Image.open("data/images/2.jpg")), np.array(image_2_2))
psnr_21_1 = peak_signal_noise_ratio(np.array(Image.open("data/images/21.png")), np.array(image_21_1))
psnr_21_2 = peak_signal_noise_ratio(np.array(Image.open("data/images/21.png")), np.array(image_21_2))
# calculate psnr for orginal image
psnr_2 = peak_signal_noise_ratio(np.array(Image.open("data/images/2.jpg")), np.array(Image.open("data/images/2.jpg")).copy())
psnr_21 = peak_signal_noise_ratio(np.array(Image.open("data/images/21.png")), np.array(Image.open("data/images/21.png")).copy())

# visualize images in 2x3 grid
fig, axes = plt.subplots(2, 3, figsize=(15, 12))
axes[0, 1].imshow(Image.open("data/images/2.jpg"))
axes[0, 1].axis('off')
axes[0, 1].set_title(f"Originalbild \nRMSE: {rmse_2:.2f}, PSNR: {psnr_2:.2f}" , fontsize=10)
axes[0, 2].imshow(image_2_1)
axes[0, 2].axis('off')
axes[0, 2].set_title(f"Pixelwerte +100\nRMSE: {rmse_2_1:.2f}, PSNR: {psnr_2_1:.2f}", fontsize=10)
axes[0, 0].imshow(image_2_2)
axes[0, 0].axis('off')
axes[0, 0].set_title(f"Pixelwerte -100\nRMSE: {rmse_2_2:.2f}, PSNR: {psnr_2_2:.2f}", fontsize=10)
axes[1, 1].imshow(Image.open("data/images/21.png"))
axes[1, 1].axis('off')
axes[1, 1].set_title(f"Originalbild \nRMSE: {rmse_21:.2f}, PSNR: {psnr_21:.2f}", fontsize=10)
axes[1, 2].imshow(image_21_1)
axes[1, 2].axis('off')
axes[1, 2].set_title(f"Pixelwerte +150\nRMSE: {rmse_21_1:.2f}, PSNR: {psnr_21_1:.2f}", fontsize=10)
axes[1, 0].imshow(image_21_2)
axes[1, 0].axis('off')
axes[1, 0].set_title(f"Pixelwerte -150\nRMSE: {rmse_21_2:.2f}, PSNR: {psnr_21_2:.2f}", fontsize=10)

plt.subplots_adjust(wspace=0.1, hspace=0.1)
sns.set_style("whitegrid")
plt.suptitle("Pixelwerte verändern", fontsize=20, y=0.9)
plt.tight_layout()
plt.show()



In [ ]:
# Histogramm der Bilder
image_and_histo(image1=np.array(Image.open("data/images/2.jpg")), 
                image2=np.array(image_2_1), 
                title1="Originalbild", 
                title2="Pixelwerte +100",
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/2.jpg")),
                image2=np.array(image_2_2),
                title1="Originalbild",
                title2="Pixelwerte -100", 
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/21.png")),
                image2=np.array(image_21_1),
                title1="Originalbild",
                title2="Pixelwerte +150",
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/21.png")),
                image2=np.array(image_21_2),
                title1="Originalbild",
                title2="Pixelwerte -150",
                log_scale=True)

#### 1.1.4.2 Experiment durch Filtern 

Die Idee in diesem Experiment ist es nach einen bestimmten Pixel Wertebereich zu filtern und anschliessend die Bilder zu visualisieren und zu vergleichen.

Auch hier schreiben wir eine Funktion, die es uns ermöglicht, die Filterung auf die einzelnen Farbkanäle anzuwenden. Um zu überprüfen, ob die Filterung funktioniert, filtern wir den Farbkanal Rot mit einem Wertebereich von 0 bis 0, also bzw eine Entfernung und erwarten ein Bild mit blau und grün tönen. 

In [ ]:
def filter_pixel_values(image_path, filter_r, filter_g, filter_b):
    # Bild einlesen
    image = Image.open(image_path)
    
    # Umwandeln in ein Numpy-Array
    img_array = np.array(image)

    # Überprüfen, ob das Bild die korrekte Form hat
    if img_array.ndim != 3 or img_array.shape[2] != 3:
        raise ValueError("Das Eingabebild muss die Form (Höhe, Breite, 3) haben.")
    
    # Pixelwerte filtern
    red_channel = np.where((img_array[:,:,0] >= filter_r[0]) & (img_array[:,:,0] <= filter_r[1]), img_array[:,:,0], 0)
    green_channel = np.where((img_array[:,:,1] >= filter_g[0]) & (img_array[:,:,1] <= filter_g[1]), img_array[:,:,1], 0)
    blue_channel = np.where((img_array[:,:,2] >= filter_b[0]) & (img_array[:,:,2] <= filter_b[1]), img_array[:,:,2], 0)
    
    # Set correct dtype for image array
    filtered_img_array = np.stack([red_channel, green_channel, blue_channel], axis=2).astype(np.uint8)
    
    # Bild aus gefiltertem Array erstellen
    filtered_image = Image.fromarray(filtered_img_array)
    
    # Return filtered image
    return filtered_image

In [ ]:
# Funktion testen indem Rot weg gefiltert wird
image_2_1_filter = filter_pixel_values(image_path='data/images/2.jpg', 
                                       filter_r = (0, 0), 
                                       filter_g = (0, 255), 
                                       filter_b = (0, 255))

image_2_2_filter = filter_pixel_values(image_path='data/images/2.jpg',
                                        filter_r = (0, 255),
                                        filter_g = (0, 0),
                                        filter_b = (0, 255))

image_21_1_filter = filter_pixel_values(image_path='data/images/21.png',
                                        filter_r = (0, 0),
                                        filter_g = (0, 255),
                                        filter_b = (0, 255))

image_21_2_filter = filter_pixel_values(image_path='data/images/21.png',
                                        filter_r = (0, 255),
                                        filter_g = (0, 0),
                                        filter_b = (0, 255))

# Calculate MSE for the images
rmse_2_1_filter = calc_rmse_images(image1 = np.array(Image.open("data/images/2.jpg")), image2 = np.array(image_2_1_filter))
rmse_2_2_filter = calc_rmse_images(image1 = np.array(Image.open("data/images/2.jpg")), image2 = np.array(image_2_2_filter))
rmse_21_1_filter = calc_rmse_images(image1 = np.array(Image.open("data/images/21.png")), image2 = np.array(image_21_1_filter))
rmse_21_2_filter = calc_rmse_images(image1 = np.array(Image.open("data/images/21.png")), image2 = np.array(image_21_2_filter))

# Calculate PSNR for the images
psnr_2_1_filter = peak_signal_noise_ratio(np.array(Image.open("data/images/2.jpg")), np.array(image_2_1_filter))
psnr_2_2_filter = peak_signal_noise_ratio(np.array(Image.open("data/images/2.jpg")), np.array(image_2_2_filter))
psnr_21_1_filter = peak_signal_noise_ratio(np.array(Image.open("data/images/21.png")), np.array(image_21_1_filter))
psnr_21_2_filter = peak_signal_noise_ratio(np.array(Image.open("data/images/21.png")), np.array(image_21_2_filter))

# visualize images in 2x3 grid
fig, axes = plt.subplots(2, 3, figsize=(15, 12))
axes[0, 1].imshow(Image.open("data/images/2.jpg"))
axes[0, 1].axis('off')
axes[0, 1].set_title(f"Originalbild \nRMSE: {rmse_2:.2f}, PSNR: {psnr_2:.2f}" , fontsize=10)
axes[0, 2].imshow(image_2_1_filter)
axes[0, 2].axis('off')
axes[0, 2].set_title(f"Rotkanal gefiltert\nRMSE: {rmse_2_1_filter:.2f}, PSNR: {psnr_2_1_filter:.2f}", fontsize=10)
axes[0, 0].imshow(image_2_2_filter)
axes[0, 0].axis('off')
axes[0, 0].set_title(f"Grünkanal gefiltert\nRMSE: {rmse_2_2_filter:.2f}, PSNR: {psnr_2_2_filter:.2f}", fontsize=10)
axes[1, 1].imshow(Image.open("data/images/21.png"))
axes[1, 1].axis('off')
axes[1, 1].set_title(f"Originalbild \nRMSE: {rmse_21:.2f}, PSNR: {psnr_21:.2f}", fontsize=10)
axes[1, 2].imshow(image_21_1_filter)
axes[1, 2].axis('off')
axes[1, 2].set_title(f"Rotkanal gefiltert\nRMSE: {rmse_21_1_filter:.2f}, PSNR: {psnr_21_1_filter:.2f}", fontsize=10)
axes[1, 0].imshow(image_21_2_filter)
axes[1, 0].axis('off')
axes[1, 0].set_title(f"Grünkanal gefiltert\nRMSE: {rmse_21_2_filter:.2f}, PSNR: {psnr_21_2_filter:.2f}", fontsize=10)

plt.subplots_adjust(wspace=0.1, hspace=0.1)
sns.set_style("whitegrid")
plt.suptitle("Rotkanal & Grünkanal filtern", fontsize=20, y=0.9)
plt.tight_layout()
plt.show()


In [ ]:
# Histogramm der Bilder
image_and_histo(image1=np.array(Image.open("data/images/2.jpg")),
                image2=np.array(image_2_1_filter),
                title1="Originalbild",
                title2="Rotkanal gefiltert",
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/2.jpg")),
                image2=np.array(image_2_2_filter),
                title1="Originalbild",
                title2="Grünkanal gefiltert",
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/21.png")),
                image2=np.array(image_21_1_filter),
                title1="Originalbild",
                title2="Rotkanal gefiltert",
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/21.png")),
                image2=np.array(image_21_2_filter),
                title1="Originalbild",
                title2="Grünkanal gefiltert",
                log_scale=True)


### 1.1.5 Experiment Schärfe

In Experiment Schärfe, versuchen wir einerseits das Bild mittels Filtern (Kernel) zu schärfen. 
Weiter versuchen wir auch, die Bilder mittels einer selbst implementierten Gausschen Weichzeichenfilter zu verschwimmen und schauen uns die Resultate dann an. 

#### 1.1.5.1 Experiment durch Filter

Wir schreiben eine Funktion, die es uns erlaubt, ein beliebiges Bild mit einem beliebigen Filter der Wahl zu filtern, um somit die Schärfe des Bildes zu erhöhen oder zu verringern und schauen, ob das Resultat unseren Erwartungen entspricht.

In [ ]:
# Implementierung der Convulution Filter
def convolve2D(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    """
    Convolves an image with a kernel.

    Parameters:
    image (np.ndarray): Input image (2D array).
    kernel (np.ndarray): Kernel (2D array).

    Returns:
    np.ndarray: Convolved image.
    """
    # Get the dimensions of the image and the kernel
    image_height, image_width = image.shape
    kernel_height, kernel_width = kernel.shape

    # Allocate space for the output image
    output = np.zeros_like(image, dtype=float)

    # Compute the padding size for the image
    pad_height = (kernel_height - 1) // 2
    pad_width = (kernel_width - 1) // 2

    # Pad the image with zeros
    padded_image = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant')

    # Perform convolution
    for y in range(image_height):
        for x in range(image_width):
            region = padded_image[y:y + kernel_height, x:x + kernel_width]
            output[y, x] = np.sum(region * kernel)

    return output

def image_sharpness(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    """
    Sharpens an image using a specified convolution kernel.
    
    Parameters:
    image (np.ndarray): The input image as a Numpy array with shape (height, width, 3).
    kernel (np.ndarray): The convolution kernel used for image sharpening.

    Returns:
    np.ndarray: The sharpened image.
    """
    # Ensure image has the correct shape
    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("The input image must have the shape (height, width, 3).")
    
    # Initialize the output image
    sharpened = np.zeros_like(image, dtype=np.float32)

    # Apply the convolution kernel to each channel separately
    for channel in range(3):
        sharpened[:, :, channel] = convolve2D(image[:, :, channel].astype(float), kernel)
    
    # Clip values to be in the valid range and convert back to uint8
    np.clip(sharpened, 0, 255, out=sharpened)
    sharpened = sharpened.astype(np.uint8)
    
    return sharpened

In [ ]:
# test functino image_sharpness
# Create a sharpening kernel
kernel = np.array([[-1, -1, -1],
                   [-1,  9, -1],
                   [-1, -1, -1]])

# Apply the kernel to the image
image_10_sharp = image_sharpness(image=np.array(Image.open("data/images/10.jpg")), 
                                 kernel=np.array([[-1, -1, -1],
                                                  [-1,  9, -1],
                                                  [-1, -1, -1]]))

image_19_sharp = image_sharpness(image=np.array(Image.open("data/images/19.jpg")),
                                    kernel=np.array([[-1, -1, -1],
                                                    [-1,  9, -1],
                                                    [-1, -1, -1]]))


In [ ]:
# Calculate MSE for the images
rmse_10_sharp = calc_rmse_images(image1 = np.array(Image.open("data/images/10.jpg")), image2 = np.array(image_10_sharp))
rmse_19_sharp = calc_rmse_images(image1 = np.array(Image.open("data/images/19.jpg")), image2 = np.array(image_19_sharp))
# calculate MSE for orginal image
rmse_10 = calc_rmse_images(image1 = np.array(Image.open("data/images/10.jpg")), image2 = np.array(Image.open("data/images/10.jpg")).copy())
rmse_19 = calc_rmse_images(image1 = np.array(Image.open("data/images/19.jpg")), image2 = np.array(Image.open("data/images/19.jpg")).copy())

# Calculate PSNR for the images
psnr_10_sharp = peak_signal_noise_ratio(np.array(Image.open("data/images/10.jpg")), np.array(image_10_sharp))
psnr_19_sharp = peak_signal_noise_ratio(np.array(Image.open("data/images/19.jpg")), np.array(image_19_sharp))
# calculate psnr for orginal image
psnr_10 = peak_signal_noise_ratio(np.array(Image.open("data/images/10.jpg")), np.array(Image.open("data/images/10.jpg")).copy())
psnr_19 = peak_signal_noise_ratio(np.array(Image.open("data/images/19.jpg")), np.array(Image.open("data/images/19.jpg")).copy())

# visualize images in 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes[0, 0].imshow(Image.open("data/images/10.jpg"))
axes[0, 0].axis('off')
axes[0, 0].set_title(f"Originalbild\nRMSE: {rmse_10:.2f}, PSNR: {psnr_10:.2f}", fontsize=10)
axes[0, 1].imshow(image_10_sharp)
axes[0, 1].axis('off')
axes[0, 1].set_title(f"Sharpness\nRMSE: {rmse_10_sharp:.2f}, PSNR: {psnr_10_sharp:.2f}", fontsize=10)

axes[1, 0].imshow(Image.open("data/images/19.jpg"))
axes[1, 0].axis('off')
axes[1, 0].set_title(f"Originalbild\nRMSE: {rmse_19:.2f}, PSNR: {psnr_19:.2f}", fontsize=10)
axes[1, 1].imshow(image_19_sharp)
axes[1, 1].axis('off')
axes[1, 1].set_title(f"Sharpness\nRMSE: {rmse_19_sharp:.2f}, PSNR: {psnr_19_sharp:.2f}", fontsize=10)

plt.subplots_adjust(wspace=0.1, hspace=0.1)
sns.set_style("whitegrid")
plt.suptitle("Bildschärfe", fontsize=20, y=1)
plt.tight_layout()
plt.show()


In [ ]:
# Histogramm der Bilder
image_and_histo(image1=np.array(Image.open("data/images/10.jpg")),
                image2=np.array(image_10_sharp),
                title1="Originalbild",
                title2="Geschärftes Bild",
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/19.jpg")),
                image2=np.array(image_19_sharp),
                title1="Originalbild",
                title2="Geschärftes Bild",
                log_scale=True)



#### 1.1.5.2 Experiment mit Gausschen Weichzeichenfilter

Auch hier schreiben wir eine Funktion, analog der vorherhigen, einfach das die Filterwerte durch eine Normalverteilung generiert werden und erhoffen uns somit ein verschwommenes Bil zu erhalten.

In [ ]:
def gaussian_kernel(size: int, sigma: float):
    """Generates a Gaussian kernel."""
    kernel = np.fromfunction(lambda x, y: (1/ (2 * np.pi * sigma ** 2)) * 
                             np.exp(- ((x - (size - 1) / 2) ** 2 + (y - (size - 1) / 2) ** 2) / (2 * sigma ** 2)),
                             (size, size))
    return kernel / np.sum(kernel)

def gaussian_filter(image: np.ndarray, sigma: float) -> np.ndarray:
    """Applies a Gaussian filter to an image."""
    size = int(6 * sigma + 1)
    size = size + 1 if size % 2 == 0 else size  # size must be odd
    kernel = gaussian_kernel(size, sigma)
    return convolve2D(image, kernel)


def blurry_image(image: np.ndarray, sigma: float) -> np.ndarray:
    """
    Blurs an image using a specified sigma value for the Gaussian filter.
    
    Parameters:
    image (np.ndarray): The input image as a Numpy array with shape (height, width, 3).
    sigma (float): The sigma value used for the Gaussian blur.

    Returns:
    np.ndarray: The blurred image.
    """
    # Ensure image has the correct shape
    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("The input image must have the shape (height, width, 3).")
    
    # Initialize the output image
    blurred = np.zeros_like(image, dtype=np.float32)

    # Apply the Gaussian filter to each channel separately
    for channel in range(3):
        blurred[:, :, channel] = gaussian_filter(image[:, :, channel].astype(float), sigma)
    
    # Clip values to be in the valid range and convert back to uint8
    np.clip(blurred, 0, 255, out=blurred)
    blurred = blurred.astype(np.uint8)
    
    return blurred

In [ ]:
# test function blurry_image
# Apply the kernel to the image
image_10_blurry = blurry_image(image=np.array(Image.open("data/images/10.jpg")), 
                               sigma=2.0)

image_19_blurry = blurry_image(image=np.array(Image.open("data/images/19.jpg")),
                               sigma=5.0)

# Calculate MSE for the images
rmse_10_blurry = calc_rmse_images(image1 = np.array(Image.open("data/images/10.jpg")), image2 = np.array(image_10_blurry))
rmse_19_blurry = calc_rmse_images(image1 = np.array(Image.open("data/images/19.jpg")), image2 = np.array(image_19_blurry))

# Calculate PSNR for the images
psnr_10_blurry = peak_signal_noise_ratio(np.array(Image.open("data/images/10.jpg")), np.array(image_10_blurry))
psnr_19_blurry = peak_signal_noise_ratio(np.array(Image.open("data/images/19.jpg")), np.array(image_19_blurry))


# visualize images in 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes[0, 0].imshow(Image.open("data/images/10.jpg"))
axes[0, 0].axis('off')
axes[0, 0].set_title(f"Originalbild\nRMSE: {rmse_10:.2f}, PSNR: {psnr_10:.2f}", fontsize=10)
axes[0, 1].imshow(image_10_blurry)
axes[0, 1].axis('off')
axes[0, 1].set_title(f"Verwischtes Bild\nRMSE: {rmse_10_blurry:.2f}, PSNR: {psnr_10_blurry:.2f}", fontsize=10)

axes[1, 0].imshow(Image.open("data/images/19.jpg"))
axes[1, 0].axis('off')
axes[1, 0].set_title(f"Originalbild\nRMSE: {rmse_19:.2f}, PSNR: {psnr_19:.2f}", fontsize=10)
axes[1, 1].imshow(image_19_blurry)
axes[1, 1].axis('off')
axes[1, 1].set_title(f"Verwischtes Bild\nRMSE: {rmse_19_blurry:.2f}, PSNR: {psnr_19_blurry:.2f}", fontsize=10)

plt.subplots_adjust(wspace=0.1, hspace=0.1)
sns.set_style("whitegrid")
plt.suptitle("Bildunschärfe", fontsize=20, y=1)
plt.tight_layout()
plt.show()


In [ ]:
# Histogramm der Bilder
image_and_histo(image1=np.array(Image.open("data/images/10.jpg")),
                image2=np.array(image_10_blurry),
                title1="Originalbild",
                title2="Verwischtes Bild")

image_and_histo(image1=np.array(Image.open("data/images/19.jpg")),
                image2=np.array(image_19_blurry),
                title1="Originalbild",
                title2="Verwischtes Bild")


### 1.1.6 Experiment Farbräume (Korrektur oder Transformationen)

In diesem Experiment versuchen wir eine Helligkeitskorrektur durchzuführen. 
Weiter untersuchen wir die das HSV-Farbmodell und schauen uns die Bilder an, ob diese denn tatsächlich Lebendiger, Reiner sind.

#### 1.1.6.1 Experiment Helligkeitskorrektur

In diesem Experiment versuchen wir die Helligkeit der Biler zu erhöhen indem wir die Bilder in den LSV-Farbraum konvertieren und anschliessend die Helligkeit (luminance) verändern un schauen uns an, was mit den Bildern geschieht.

In [ ]:
def adjust_brightness_lab(image: np.ndarray, factor: float) -> np.ndarray:
    """
    Adjusts the brightness of an image in the LAB color space.

    Parameters:
    image (np.ndarray): The input image as a Numpy array with shape (height, width, 3).
    factor (float): The factor by which to adjust the brightness in the L channel.
                    A factor of 1.0 means no change. Values can be above or below 1 to increase or decrease brightness, respectively.

    Returns:
    np.ndarray: The image with adjusted brightness.
    """
    # Ensure image has the correct shape and normalize to range [0, 1]
    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("The input image must have the shape (height, width, 3).")
    image = image / 255.0 if image.dtype == np.uint8 else image
    
    # Convert image to LAB color space
    lab_image = color.rgb2lab(image)
    
    # Adjust the L channel by the specified factor
    lab_image[:, :, 0] = np.clip(lab_image[:, :, 0] * factor, 0, 100)
    
    # Convert back to RGB color space
    rgb_image = color.lab2rgb(lab_image)
    
    # If the original image was uint8, convert back to uint8
    if image.dtype == np.uint8:
        rgb_image = (rgb_image * 255).astype(np.uint8)
    
    return rgb_image


In [ ]:
# test function adjust_brightness_lab
image_11_bright = adjust_brightness_lab(image=np.array(Image.open("data/images/11.jpg")), 
                                        factor=1.5)

image_22_bright = adjust_brightness_lab(image=np.array(Image.open("data/images/22.png")),
                                        factor=0.5)

# Calculate MSE for the images
rmse_11_bright = calc_rmse_images(image1 = np.array(Image.open("data/images/11.jpg")), image2 = np.array(image_11_bright))
rmse_22_bright = calc_rmse_images(image1 = np.array(Image.open("data/images/22.png")), image2 = np.array(image_22_bright))
# calculate MSE for orginal image
rmse_11 = calc_rmse_images(image1 = np.array(Image.open("data/images/11.jpg")), image2 = np.array(Image.open("data/images/11.jpg")).copy())
rmse_22 = calc_rmse_images(image1 = np.array(Image.open("data/images/22.png")), image2 = np.array(Image.open("data/images/22.png")).copy())

# Calculate PSNR for the images
psnr_11_bright = peak_signal_noise_ratio(np.array(Image.open("data/images/11.jpg")), np.array(image_11_bright))
psnr_22_bright = peak_signal_noise_ratio(np.array(Image.open("data/images/22.png")), np.array(image_22_bright))
# calculate psnr for orginal image
psnr_11 = peak_signal_noise_ratio(np.array(Image.open("data/images/11.jpg")), np.array(Image.open("data/images/11.jpg")).copy())
psnr_22 = peak_signal_noise_ratio(np.array(Image.open("data/images/22.png")), np.array(Image.open("data/images/22.png")).copy())

# visualize images in 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes[0, 0].imshow(Image.open("data/images/11.jpg"))
axes[0, 0].axis('off')
axes[0, 0].set_title(f"Originalbild\nRMSE: {rmse_11:.2f}, PSNR: {psnr_11:.2f}", fontsize=10)
axes[0, 1].imshow(image_11_bright)
axes[0, 1].axis('off')
axes[0, 1].set_title(f"Helleres Bild\nRMSE: {rmse_11_bright:.2f}, PSNR: {psnr_11_bright:.2f}", fontsize=10)

axes[1, 0].imshow(Image.open("data/images/22.png"))
axes[1, 0].axis('off')
axes[1, 0].set_title(f"Originalbild\nRMSE: {rmse_22:.2f}, PSNR: {psnr_22:.2f}", fontsize=10)
axes[1, 1].imshow(image_22_bright)
axes[1, 1].axis('off')
axes[1, 1].set_title(f"Helleres Bild\nRMSE: {rmse_22_bright:.2f}, PSNR: {psnr_22_bright:.2f}", fontsize=10)

plt.subplots_adjust(wspace=0.1, hspace=0.1)
sns.set_style("whitegrid")
plt.suptitle("Helligkeit", fontsize=20, y=1)
plt.tight_layout()
plt.show()


In [ ]:
# Histogramm der Bilder
image_and_histo(image1=np.array(Image.open("data/images/11.jpg")),
                image2=np.array(image_11_bright),
                title1="Originalbild",
                title2="Helleres Bild",
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/22.png")),
                image2=np.array(image_22_bright),
                title1="Originalbild",
                title2="Dunkleres Bild",
                log_scale=True)



#### 1.1.6.2 Experiment HSV-Farbmodell

In diesem Experiment spielen wir mit den Werten von V im HSV-Farbmodell, indem wir einen Histogramm-Equalizer auf den V-Kanal anwenden und schauen uns die Resultate an.

In [ ]:
def adjust_contrast_hsv(image: np.ndarray) -> np.ndarray:
    """
    Adjusts the brightness of an image in the HSV color space using histogram equalization.

    Parameters:
    image (np.ndarray): The input image as a Numpy array with shape (height, width, 3).

    Returns:
    np.ndarray: The image with adjusted brightness.
    """
    # Ensure image has the correct shape
    if image.ndim != 3 or image.shape[2] != 3:
        raise ValueError("The input image must have the shape (height, width, 3).")

    # Convert image to HSV color space
    hsv_image = color.rgb2hsv(image)
    
    # Apply histogram equalization to the V channel
    hsv_image[:, :, 2] = exposure.equalize_hist(hsv_image[:, :, 2], nbins=256, mask=1)
    
    # Convert back to RGB color space
    rgb_image = color.hsv2rgb(hsv_image)
    
    # Convert back to uint8 if necessary
    if image.dtype == np.uint8:
        rgb_image = (rgb_image * 255).astype(np.uint8)
    
    return rgb_image

In [ ]:
# test function adjust_brightness_lab
image_11_bright = adjust_contrast_hsv(image=np.array(Image.open("data/images/11.jpg")))
image_22_bright = adjust_contrast_hsv(image=np.array(Image.open("data/images/22.png")))

# Calculate MSE for the images
rmse_11_bright = calc_rmse_images(image1 = np.array(Image.open("data/images/11.jpg")), image2 = np.array(image_11_bright))
rmse_22_bright = calc_rmse_images(image1 = np.array(Image.open("data/images/22.png")), image2 = np.array(image_22_bright))

# Calculate PSNR for the images
psnr_11_bright = peak_signal_noise_ratio(np.array(Image.open("data/images/11.jpg")), np.array(image_11_bright))
psnr_22_bright = peak_signal_noise_ratio(np.array(Image.open("data/images/22.png")), np.array(image_22_bright))

# visualize images in 2x2 grid
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes[0, 0].imshow(Image.open("data/images/11.jpg"))
axes[0, 0].axis('off')
axes[0, 0].set_title(f"Originalbild\nRMSE: {rmse_11:.2f}, PSNR: {psnr_11:.2f}", fontsize=10)
axes[0, 1].imshow(image_11_bright)
axes[0, 1].axis('off')
axes[0, 1].set_title(f"Kontraständerung\nRMSE: {rmse_11_bright:.2f}, PSNR: {psnr_11_bright:.2f}", fontsize=10)

axes[1, 0].imshow(Image.open("data/images/22.png"))
axes[1, 0].axis('off')
axes[1, 0].set_title(f"Originalbild\nRMSE: {rmse_22:.2f}, PSNR: {psnr_22:.2f}", fontsize=10)
axes[1, 1].imshow(image_22_bright)
axes[1, 1].axis('off')
axes[1, 1].set_title(f"Kontraständerung\nRMSE: {rmse_22_bright:.2f}, PSNR: {psnr_22_bright:.2f}", fontsize=10)

plt.subplots_adjust(wspace=0.1, hspace=0.1)
sns.set_style("whitegrid")
plt.suptitle("Kontraständerung", fontsize=20, y=1)
plt.tight_layout()
plt.show()


In [ ]:
# Histogramm der Bilder
image_and_histo(image1=np.array(Image.open("data/images/11.jpg")),
                image2=np.array(image_11_bright),
                title1="Originalbild",
                title2="Kontraständerung",
                log_scale=True)

image_and_histo(image1=np.array(Image.open("data/images/22.png")),
                image2=np.array(image_22_bright),
                title1="Originalbild",
                title2="Kontraständerung",
                log_scale=True)


## 1.2 Signaleigenschaften
Suche verschiedene Signale unterschiedlicher Szenen passend zu deinem Land oder nehme selbst welche auf. Die Signale sollen sich eignen, um Anpassungen der Signaleigenschaften {'Bandbreite, Phase, Rauschen'} in Experimenten zu demonstrieren. Messe dazu zuerst auf deinen Signalen die dir zugeordneten Signaleigenschaften mittels geeigneten Metriken. Definiere danach ein Ziel, wie die dir zugeordneten Signaleigenschaften verändert werden sollen. Definiere ein paar Experimente, um dieses Ziel zu erreichen, und führe die Experimente mit deinen Signalen und geeigneten Methoden aus. Demonstriere mit deinen aufgenommenen Signale das Nyquist Theorem. Diskutiere deine Daten-, Parameter- und Methodenwahl und die erzielten Ergebnisse. 

In [ ]:
# Signal Daten einlesen
from IPython.display import Audio
# Konvertierung
from pydub import AudioSegment
# Librosa
import librosa

# soundfile
import soundfile as sf

# wavfile einlesen
from scipy.io import wavfile



### 1.2.1 Signale Laden

Auch bei den Signal Daten haben wir ein Pool von fünf Daten die wir für unsere Experimente verwenden können. 

In [ ]:
audio_files = ["1.wav", "2.wav", "3.wav", "4.wav", "5.wav"]

for file in audio_files:
    print(f"Signal: {file}")
    display(Audio(filename=f"data/signals/{file}"))

### 1.2.2 Signale selektieren & geeignete Metriken

Eine Eigenschaft von Signalen die wir visualisieren können ist das Amplitude-Frequenz-Spektrum, welches wir als Funktion hinschreiben. 

In [ ]:
# Create a function to plot the signal amplitude vs time
def plot_signal(file_path, sample_rate=44100):
    # Load audio file
    audio, _ = librosa.load(file_path, sr=sample_rate)

    # Create time array
    time = np.arange(0, len(audio))/sample_rate

    # Plot the signal
    plt.figure(figsize=(15, 5))
    plt.plot(time, audio)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude")
    plt.show()
    

# Createa a function to plot the spectrum
def plot_single_spectrum(file_path, title):
    
    # Load the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    
    # Calculate the spectrum
    fft_result = np.fft.fft(audio_data)
    frequencies = np.fft.fftfreq(len(fft_result), 1.0 / sample_rate)

    # Visualize the spectrum
    plt.figure(figsize=(12, 4))
    plt.plot(frequencies, np.abs(fft_result))
    plt.title(title)
    plt.xlabel("Frequenz (Hz)")
    plt.ylabel("Amplitude")
    plt.show()

    # Create a function thath uses both function above to visualize the spectrum next to each other 
def plot_signal_and_spectrum(file_path, title):
    # Load the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    
    # Create time array
    time = np.arange(0, len(audio_data))/sample_rate
    
    # Calculate the spectrum
    fft_result = np.fft.fft(audio_data)
    frequencies = np.fft.fftfreq(len(fft_result), 1.0 / sample_rate)

    # Visualize the signal and the spectrum
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    axes[0].plot(time, audio_data)
    axes[0].set_xlabel("Time (seconds)")
    axes[0].set_ylabel("Amplitude")
    axes[0].set_title("Visualization of Recording")
    axes[1].plot(frequencies, np.abs(fft_result))
    axes[1].set_xlabel("Frequenz (Hz)")
    axes[1].set_ylabel("Amplitude")
    axes[1].set_title("Visualization of Spectrum")
    fig.suptitle(title)
    plt.show()

In [ ]:
def plot_compare_signals_and_spectrums(file_path1, file_path2, title_1, title_2):
    # Load the first audio file
    audio_data1, sample_rate1 = librosa.load(file_path1, sr=None)
    
    # Calculate the spectrum for the first audio file
    fft_result1 = np.fft.fft(audio_data1)
    frequencies1 = np.fft.fftfreq(len(fft_result1), 1.0 / sample_rate1)
    
    # Time for the first audio file
    time1 = np.arange(0, len(audio_data1))/sample_rate1
    
    # Load the second audio file
    audio_data2, sample_rate2 = librosa.load(file_path2, sr=None)
    
    # Calculate the spectrum for the second audio file
    fft_result2 = np.fft.fft(audio_data2)
    frequencies2 = np.fft.fftfreq(len(fft_result2), 1.0 / sample_rate2)
    
    # Time for the second audio file
    time2 = np.arange(0, len(audio_data2))/sample_rate2
    
    # Create a figure with 2x2 subplots
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot the first signal
    axs[0, 0].plot(time1, audio_data1)
    axs[0, 0].set_title(f'Signal of {title_1}')
    axs[0, 0].set_xlabel('Time (seconds)')
    axs[0, 0].set_ylabel('Amplitude')
    
    # Plot the second signal
    axs[1, 0].plot(time2, audio_data2)
    axs[1, 0].set_title(f'Signal of {title_2}')
    axs[1, 0].set_xlabel('Time (seconds)')
    axs[1, 0].set_ylabel('Amplitude')
    
    # Plot the first spectrum
    axs[0, 1].plot(frequencies1, np.abs(fft_result1))
    axs[0, 1].set_title(f'Spectrum of {title_1}')
    axs[0, 1].set_xlabel('Frequency (Hz)')
    axs[0, 1].set_ylabel('Amplitude')
    
    # Plot the second spectrum
    axs[1, 1].plot(frequencies2, np.abs(fft_result2))
    axs[1, 1].set_title(f'Spectrum of {title_2}')
    axs[1, 1].set_xlabel('Frequency (Hz)')
    axs[1, 1].set_ylabel('Amplitude')
    
    # Adjust spacing and display plot
    plt.tight_layout()
    plt.show()


In [ ]:
def plot_compare_spectrums(file_path1, file_path2, title_1, title_2):
    # Load the first audio file
    audio_data1, sample_rate1 = librosa.load(file_path1, sr=None)

    # Calculate the spectrum for the first audio file
    fft_result1 = np.fft.fft(audio_data1)
    frequencies1 = np.fft.fftfreq(len(fft_result1), 1.0 / sample_rate1)

    # Load the second audio file
    audio_data2, sample_rate2 = librosa.load(file_path2, sr=None)

    # Calculate the spectrum for the second audio file
    fft_result2 = np.fft.fft(audio_data2)
    frequencies2 = np.fft.fftfreq(len(fft_result2), 1.0 / sample_rate2)

    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(25, 10))

    # Plot the first spectrum
    ax1.plot(frequencies1, np.abs(fft_result1))
    ax1.set_title(title_1)
    ax1.set_xlabel("Frequency (Hz)")
    ax1.set_ylabel("Amplitude")

    # Plot the second spectrum
    ax2.plot(frequencies2, np.abs(fft_result2))
    ax2.set_title(title_2)
    ax2.set_xlabel("Frequency (Hz)")
    ax2.set_ylabel("Amplitude")

    plt.tight_layout()  # Ensure proper spacing between subplots
    plt.show()

### 1.2.3 Ziel definieren


| Experiment | Ziel | Signal |
| --- | --- | --- |
| Bandbreite | Das Ziel ist es die Brandbreite eines Signals beliebig filtern zu können  | 5.wav |
| Phase | Eine Phasenverschiebung der Signale | 5.wav |
| Rauschen |  |  | 

### 1.2.4 Experiment Bandbreite

#### 1.2.4.1 Experiment durch Filtern

In [ ]:
def filter_frequency(file_path, filter_range=[0, None], output_file="filtered_audio.wav", export=False):
    """
    Load an audio file, compute its spectrum, filter it by frequency,
    visualize the filtered spectrum, and save the filtered audio.

    Parameters:
    file_path (str): path to the audio file.
    filter_range (list): list of two elements defining the start and end of the filter frequency range.
    output_file (str): path to save the filtered audio file.
    """
    # Load the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    
    # Compute the Fast Fourier Transform (FFT) of the audio data
    fft_result = np.fft.fft(audio_data)
    frequencies = np.fft.fftfreq(len(fft_result), 1.0 / sample_rate)

    # Filter the FFT result
    if filter_range[1] is None:
        filter_range[1] = np.max(frequencies)
    
    filter_mask = (frequencies >= filter_range[0]) & (frequencies <= filter_range[1]) | \
                  (frequencies >= -filter_range[1]) & (frequencies <= -filter_range[0])
                  
    filtered_fft = fft_result.copy()
    filtered_fft[~filter_mask] = 0  # Set the amplitudes outside the filter range to zero
    
    # Inverse FFT to convert back to time domain
    filtered_audio = np.fft.ifft(filtered_fft).real

    # if export is True, save the filtered audio
    if export:
        sf.write(output_file, filtered_audio, sample_rate)

# Use function to filter audio signal in range [500, 1000] Hz
filter_frequency("data/signals/5.mp3", 
                 filter_range=[250, None], 
                 output_file="data/signals/filtered_5.wav", 
                 export=True)


In [ ]:
signal_experi1_files = ["5.wav", "filtered_5.wav"]

for file in signal_experi1_files:
    print(file)
    display(Audio(filename=f"data/signals/{file}"))

# Plot the spectrum
plot_compare_signals_and_spectrums(file_path1='data/signals/5.wav',
               file_path2='data/signals/filtered_5.wav',
               title_1="Spectrum of 5.wav",
               title_2="Spectrum of filtered_5.wav")


### 1.2.5 Experiment Phase



#### 1.2.5.1 Experiment Phasenverschiebung

In [ ]:
def shift_phase(mp3_file_path, phase_shift, export=False, output_wav_file=None):
    """
    Shifts the phase of an audio file and exports the result if necessary.

    Parameters:
    mp3_file_path (str): Path to the input mp3 file.
    phase_shift (float): Phase shift to be applied.
    export (bool): Flag to determine whether to export the shifted audio.
    output_wav_file (str): Path to the output wav file if export is True.
    """
    try:
        # Load the audio file
        y, sr = librosa.load(mp3_file_path)

        # Compute the phase information
        phase = librosa.stft(y)

        # Shift the phase
        phase_shifted = np.angle(phase) + phase_shift

        # Inverse STFT to get the shifted audio
        audio_shifted = librosa.istft(np.abs(phase) * np.exp(1j * phase_shifted))

        # If export is True, save the shifted audio
        if export and output_wav_file:
            sf.write(output_wav_file, audio_shifted, sr)

    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Use function to shift phase of audio signal by pi/2
shift_phase(mp3_file_path="data/signals/5.mp3", 
            phase_shift=np.pi / 2, 
            export=True, 
            output_wav_file="data/signals/phase_shifted_5.wav")

In [ ]:
def compare_shift_phase(file_path1, file_path2, title_1, title_2):
    """
    Compares the phase information of two audio files visually.

    Parameters:
    file_path1 (str): Path to the first audio file.
    file_path2 (str): Path to the second audio file.
    title_1 (str): Title for the first plot.
    title_2 (str): Title for the second plot.
    """
    try:
        # Load the audio files
        y1, sr1 = librosa.load(file_path1)
        y2, sr2 = librosa.load(file_path2)

        # Compute the phase information
        phase1 = librosa.stft(y1)
        phase2 = librosa.stft(y2)

        # Create comparison plots
        plt.figure(figsize=(20, 6))

        plt.subplot(1, 2, 1)
        librosa.display.specshow(librosa.amplitude_to_db(np.abs(phase1), ref=np.max), y_axis='log', x_axis='time')
        plt.colorbar(format='%+2.0f dB')
        plt.title(title_1)

        plt.subplot(1, 2, 2)
        librosa.display.specshow(librosa.amplitude_to_db(np.abs(phase2), ref=np.max), y_axis='log', x_axis='time')
        plt.colorbar(format='%+2.0f dB')
        plt.title(title_2)

        plt.show()

    except Exception as e:
        print(f"An error occurred: {str(e)}")


In [ ]:
signal_experi2_files = ["5.wav", "phase_shifted_5.wav"]

for file in signal_experi2_files:
    print(file)
    display(Audio(filename=f"data/signals/{file}"))

# Use function to compare phase of audio signal before and after phase shift
compare_shift_phase(file_path1="data/signals/5.mp3",
                    file_path2="data/signals/phase_shifted_5.wav",
                    title_1="Phase of 5.mp3",
                    title_2="Phase of phase_shifted_5.wav")

# Plot the spectrum
plot_compare_signals_and_spectrums(file_path1='data/signals/5.wav',
               file_path2='data/signals/phase_shifted_5.wav',
               title_1="Spectrum of 5.wav",
               title_2="Spectrum of phase_shifted_5.wav")

### 1.2.6 Experiment Rauschen

Wir versuchen aus dem Audio Signal das Rauschen zu minimieren indem wir einen Filter anwenden. 

#### 1.2.6.1 Experiment 1-D Filter


In [ ]:
def smooth_signal(file_path, filter, mode='valid', export=False, output_wav_file=None):
    """
    Smooths a signal using 1D convolution.

    Parameters:
    signal (array-like): The input signal to be smoothed.
    kernel (array-like): The convolution kernel.

    Returns:
    array: The smoothed signal.
    """
    # Load audio with librosa
    audio, sr = librosa.load(file_path)

    # make sure the signal is a numpy array
    filter = np.array(filter)
    
    # Apply convolution
    smoothed_signal = np.convolve(audio, 
                                  filter,
                                  mode)
    # If export is True, save the smoothed audio
    if export and output_wav_file:
        sf.write(output_wav_file, smoothed_signal, sr)

# Use function to smooth audio signal
smooth_signal(file_path="data/signals/5.mp3", 
              filter=[1/10] * 10, 
              export=True, 
              output_wav_file="data/signals/smoothed_5.wav")

In [ ]:
signal_experi3_files = ["5.wav", "smoothed_5.wav"]

for file in signal_experi3_files:
    print(file)
    display(Audio(filename=f"data/signals/{file}"))

# Plot the spectrum
plot_compare_signals_and_spectrums(file_path1='data/signals/5.wav',
               file_path2='data/signals/smoothed_5.wav',
               title_1="Spectrum of 5.wav",
               title_2="Spectrum of smoothed_5.wav")

## 1.3 Normalisierung und Standardisierung
Normalisiere und standardisiere deine Bild- und Signaldaten. Demonstriere, dass die Daten nach der Normalisierung/Standardisierung weiterhin den gleichen Inhalt zeigen. Diskutiere deine Methodenwahl und die erzielten Ergebnisse

# 2 Faltung/Filterung in Bild und Signal (LE2)

## 2.1. Filterung in der räumlichen Domäne
Implementiere selbst einen klassischen Algorithmus zur Filterung von Signalen (1D) und Bildern (2D) in der räumlichen Domäne (Faltung/Convolution). Welche Elemente soll ein solcher Algorithmus enthalten? Teste deine Funktion mit jeweils einem Signal und einem Bild passend zu deinem Land und zwei Faltungskernels in geeigneter Grösse. Die beiden Faltungskernel sollen das Signal bzw. Bild {'weichzeichnen, entrauschen'}. Skaliert dein Algorithmus für grosse Daten? Messe die Unterschiede deiner Beispielsignale/-bilder vor und nach dem Filtern mittels einer geeigneten Metrik. Diskutiere deine Daten-, Methoden- und Parameterwahl sowie die erzielten Ergebnisse. Weshalb hast du diese Faltungskernel bzw. diese Metrik gewählt? 


## 2.2. Filterung in der spektralen Domäne
Implementiere eine Methode basierend auf spektraler Filterung, welche ein repetitives Muster aus einem Signal bzw. einem Bild filtert. Die Methode soll für Signale (1D) sowie Bilder (2D) funktionieren. Demonstriere das Resultat anhand eines Signales und eines Bildes passend zu deinem Land. Diskutiere deine Daten-, Methoden- und Parameterwahl sowie die erzielten Ergebnisse. 

## 2.3. Bonus: Filterung in der räumlichen und spektralen Domäne
Führe optional Experimente mit Signalen oder Bildern mit Methoden durch, welche das Filtern in der räumlichen und der spektralen Domäne ermöglichen. Z.B. mittels Wavelets. Diskutiere deine Daten-, Parameter- und Methodenwahl und die erzielten Ergebnisse.

## 2.4. Algorithmen zur Erkennung von Strukturen in Bildern
Zeige die einzelnen Schritte eines bekannten klassischen Bildverarbeitungs-Algorithmus zur Detektion von {'Kreisen'} in einem geeigneten Beispielbild deines Landes. Die einzelnen Schritte können selbst programmiert sein oder von Bibliotheken verwendet werden. Wichtig ist, dass Zwischenergebnisse der Schritte ersichtlich sind. Beschreibe zudem, was in jedem einzelnen Schritt konzeptionell relevant ist. Diskutiere deine Daten-, Methoden- und Parameterwahl sowie die erzielten Ergebnisse. Für welche Fälle funktioniert dein Algorithmus gut bzw. schlecht? Warum hast du dieses Bild gewählt?

# 3 Peer-Grading
Nach Abgabe der Mini-Challenge hast du 1 Woche Zeit eine dir zugeordnete Abgabe von anderen zu bewerten. Die Zuordnung erfolgt via FHNW Peer-Grading-Tool (siehe Link unten). Orientiere dich für die Bewertung an den vorgegebenen Bewertungskriterien (siehe Excel-Datei oder Peer-Grading-Tool). Die Note 5 bedeutet, dass alles erfüllt ist, wie du es von einem guten Data Scientist in der Praxis erwarten würdest. Du startest als Baseline mit der Note 5. Entdeckst du Fehler, geht die Note nach unten. Der Note 5.5 nähert man sich, wenn die Erwartungen übertroffen wurden. Der Note 6 nähert man sich, wenn die Leistung ausserordentlich oder mit einer unglaublichen Eigenleistung verbunden ist bzw. es gar nicht mehr besser geht. Kritisches Denken, Vielfalt von Experimenten und eigene Ideen werden positiv bewertet. Siehe auch Checkliste für Bewertung. Die Benotung soll auf Zehntel gerundet sein. Wer auf Zehntel gerundet mit 0.1 Abweichung die Endnote von der Fachexpertin trifft, kriegt einen Bonus von 0.2 Noten für die eigene Abgabe. In der Sprechstunde vom 7. November widmen wir uns dem Peer-Grading. 
FHNW Peer-Grading-Tool
